## RFM(Recency, Frequency, Monetary)

* RFM은 가치있는 고객을 추출해내어 이를 기준으로 고객을 분류할 수 있는 매우 간단하면서도 유용하게 사용될 수 있는 방법으로 알려져 있어 마케팅에서 가장 많이 사용되고 있는 분석방법 중 하나이다. RFM은 구매 가능성이 높은 고객을 선정하기 위한 데이터 분석방법으로서, 분석과정을 통해 데이터는 의미있는 정보로 전환된다.

* RFM은 Recency, Frequency, Monetary의 약자로 고객의 가치를 다음의 세 가지 기준에 의해 계산하고 있다.

* Recency - 거래의 최근성: 고객이 얼마나 최근에 구입했는가?
* Frequency - 거래빈도: 고객이 얼마나 빈번하게 우리 상품을 구입했나?
* Monetary - 거래규모: 고객이 구입했던 총 금액은 어느 정도인가?

### RFM 모형

* Scoring 기법: RFM의 요인을 각각 5등급으로 등간격으로 분류하는 방법이다.
* 현재 개발된 RFM 모형은 크게 4가지로 분류 할 수 있다. 이 문서의 내용들은 정설이 아니며 신뢰하기 어려움을 전제로 참고해야한다.
    * 모델1. RFM 각 요소의 20% rule의 적용
    * 모델2. 비율 척도에 의한 양적인 정도의 차이에 따른 등간격의 5등급 분류
    * 모델3. 상하 20%를 제외한 등간격 척도에 의한 그룹 분류
    * 모델4. 군집 분석에 의한 각 요소 별 5개의 그룹 분류
* Data Mining 기법을 이용한 모형
* 회귀분석
* 선형 회귀 분석을 이용한 모형: 고객의 구매 최근성, 구매 빈도, 구매 금액 등 고객의 수익 기여도를 나타내는 세가지 지표들의 선형결합으로 세가지 지표들을 점수화 한다.
* 다중 회귀 분석을 이용한 모형: 각 고객의 구매 행동을 나타내는 R,F,M의 변수들을 독립변수로하고 고객의 미래 구매 행동을 예측하는 기법.
* 신경망을 적용한 모형: 로지스틱 회귀 모형을 보완하는 차원에서 연구.
* 확률적 RFM모형:Colombo와 Weina의 확률적인 RFM모형은 과거의 고객의 응답 이력으로 고객의 미래 응답을 예측하는 행동모델이다.

출처 : [RFM - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/RFM)




### 파레토 법칙

* 상위 고객의 20%가 기업 총 매출의 80%를 차지한다. 
* 80:20 법칙을 고려하면 고객 매출 데이터가 누적됨에 따라 마케팅에 데이터를 활용해 마케팅에 활용할 수 있다.
    
    
### 고객 세분화

* 상품과 서비스 판매정보 - 상품판매 자료 거래 금액, 횟수
* 인구통계학적 정보 - 나이, 성별, 직업, 학력, 거주지역, 소득수준
* 라이프 스타일 정보 - 순차적, 구매 정보, RFM 정보
* 심리 정보 - 구매욕구
* 행동 정보 - 구매패턴 Life Time Value
    
    
### RFM 관련 논문

[국회도서관 소장자료 -RFM을 적용한 고객 세분화 개선 및 우수고객 예측 모형에 관한 연구](http://dl.nanet.go.kr/law/SearchDetailView.do?cn=KDMT1200458026#none)

## 라이브러리 로드

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

## 폰트설정

In [ ]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux
        !apt-get update -qq
        !apt-get install fonts-nanum -qq  > /dev/null

        import matplotlib.font_manager as fm

        fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
        font = fm.FontProperties(fname=fontpath, size=9)
        fm._rebuild()
        font_family = "NanumBarunGothic"
    return font_family

plt.rc("font", family=get_font_family())
plt.rc("axes", unicode_minus=False)

## 데이터 로드

In [ ]:
# raw = pd.read_excel("http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx")
# raw = pd.read_excel("Online Retail.xlsx", engine="openpyxl")
raw = pd.read_csv("online_retail.csv")
raw.shape

In [ ]:
raw.head()

## 유효 데이터만 추출

In [ ]:
# "CustomerID" 가 있고 "Quantity" 가 0보다 큰 데이터를 가져옵니다.
# 구매하고 취소한 건 중 취소한 건만 제외하고 구매 건은 남깁니다.
raw_valid = raw[raw["CustomerID"].notnull() & (raw["Quantity"] > 0) & (raw["UnitPrice"] > 0)].copy()
raw_valid.shape

In [ ]:
raw_valid.head(1)

## 구매 금액 계산

In [ ]:
# TotalPrice 는 RFM 중 MonetaryValue의 값이 됩니다.
raw_valid["TotalPrice"] = raw_valid["Quantity"] * raw_valid["UnitPrice"]

In [ ]:
# TotalPrice 의 기술통계값을 봅니다.
raw_valid["TotalPrice"].describe()

## 이상치 제거

In [ ]:
# 범위를 설정하여 이상치를 찾습니다.
# 160000 보다 큰 값을 찾습니다.
raw_valid[raw_valid["TotalPrice"] > 160000]

In [ ]:
# 이상치는 제외합니다.
# 160000 보다 큰 값을 제외합니다.
raw_valid = raw_valid[raw_valid["TotalPrice"] < 160000]
raw_valid.shape

## 중복 데이터 확인
* 중복 데이터는 여러 이유로 발생할 수 있습니다. 네트워크 통신 문제나 쇼핑몰의 UI 구성 등의 다양한 이슈로 발생할 수 있습니다.
* 예를 들면 주문을 하고 새로고침을 했을 때 다시 주문 트랜잭션에 들어가거나 하는 이슈가 발생할 가능성도 있습니다.

In [ ]:
# 중복 데이터 중 첫번째 것만(keep='first') 봅니다.
# keep은 중복 데이터 발견시에 어떤 데이터를 유지하고 제외할지 결정해주는 변수입니다. ('first', 'last','False'로 선택적 입력 가능)
raw_valid[raw_valid.duplicated(keep='first')].sort_index()

In [ ]:
# 중복데이터 모두 출력(keep = False) 합니다.
raw_valid[raw_valid.duplicated(keep=False)].sort_values(
    by=["InvoiceNo", "StockCode", "CustomerID", "TotalPrice"])

## 중복 데이터 제거

In [ ]:
# drop_duplicates로 중복을 제거합니다.
print(raw_valid.shape)
df = raw_valid.______________().copy()
df.shape

## RFM 계산
### 전체 주문에서 최근 주문일 구하기

In [ ]:
# InvoiceDate를 날짜형식으로 변환해 줍니다.
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
# Recency 계산을 위해 해당 주문에서 가장 최근 구매가 일어난 시간을 가져옵니다.
# 최근 거래 기준일(last_timestamp)을 만들기 위해 timedelta로 날짜를 더해줍니다.(최소값:1 설정) 
last_timestamp = df['InvoiceDate']._____() + dt._____(days = 1)
last_timestamp

### 고객별 Recency, Frequency, Monetary 값 구하기

In [ ]:
# RFM 값을 정의합니다.
# Recency : 최근 거래 기준일(last_timestamp)과 고객별 최근 구매한 날짜(x.max())와 차이값
# Frequency : 구매 빈도수
# Monetary : 총 구매 금액
rfm = df.groupby(['CustomerID']).agg({'InvoiceDate': lambda x : (last_timestamp - x.max()).days,
                                      'InvoiceNo':'_____',
                                      'TotalPrice': '_____'})
rfm.shape

In [ ]:
rfm

In [ ]:
# RFM 으로 변수의 이름을 변경합니다.
# InvoiceDate 는 Recency
# InvoiceNo 는 Frequency
# TotalPrice 는 MonetaryValue
rfm.rename(columns={'InvoiceDate':'Recency', 
                    'InvoiceNo':'Frequency', 
                    'TotalPrice':'MonetaryValue'},
           inplace= True)
rfm

In [ ]:
rfm.describe()

## RFM 모형

* Scoring 기법: RFM의 요인을 각각 5등급으로 등간격으로 분류하는 방법이다.
* 현재 개발된 RFM 모형은 크게 4가지로 분류 할 수 있다. 이 문서의 내용들은 정설이 아니며 신뢰하기 어려움을 전제로 참고해야한다.
    * 모델1. RFM 각 요소의 20% rule의 적용
    * 모델2. 비율 척도에 의한 양적인 정도의 차이에 따른 등간격의 5등급 분류
    * 모델3. 상하 20%를 제외한 등간격 척도에 의한 그룹 분류
    * 모델4. 군집 분석에 의한 각 요소 별 5개의 그룹 분류
    
출처 : [RFM - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/RFM)

### qcut을 통한 RFM 변수 만들기

<img src="https://t1.daumcdn.net/cfile/tistory/99DEAF3B5C1E4AAA1E">

In [ ]:
# Recency 는 최근일수록 높은 스코어를 갖도록 합니다.
# Frequency, MonetaryValue 는 값이 클 수록 높은 스코어를 갖도록 합니다.
r_labels = ________
f_labels = ________
m_labels = ________
cut_size = ?

In [ ]:
r_cut = pd.________(rfm['Recency'], q=________, labels = ________)
f_cut = pd.________(rfm['Frequency'], q=________, labels = ________)
m_cut = pd.________(rfm['MonetaryValue'], q=________, labels = ________)
# assign 을 사용하면 여러 변수를 한 번에 만들 수 있습니다.
rfm = rfm._______(R=r_cut, F=f_cut, M=m_cut)

In [ ]:
rfm

In [ ]:
sns.displot(data=rfm, x="Recency", hue="R", kind="kde")

### RFM Segment

In [ ]:
# RFM_segment 값을 구합니다. 
# R, F, M 값을 문자 그대로 붙여서 세그먼트를 구분합니다.
# rfm["RFM_segment"]

### RFM Score

In [ ]:
# R, F, M 값의 합계를 구해서 세그먼트 점수를 구합니다.
# rfm["RFM_score"]

In [ ]:
# RFM segment 값에 따라 정렬합니다.
plt.figure(figsize=(20, 4))
plt.xticks(rotation=90)
sns.barplot(data=rfm.sort_values("RFM_segment"), x="RFM_segment", y="RFM_score")

In [ ]:
ax = plt.axes(projection='3d')
ax.scatter3D(rfm["R"], rfm["F"], rfm["M"])

In [ ]:
ax = plt.axes(projection='3d')
ax.scatter3D(rfm["Recency"], rfm["Frequency"], rfm["MonetaryValue"])

In [ ]:
# 문자열의 format 함수를 사용하여 소수점 아래는 표기하지 않도록({:,.0f}) 문자열 포맷을 지정합니다.
score_rfm = rfm.groupby(["RFM_score"]).agg({"Recency" : "____", 
                                "Frequency" : "____", 
                                "MonetaryValue" : ["____", "____"]})
score_rfm.style.background_gradient().format("{:,.0f}")

### qcut을 통한 고객군 나누기

In [ ]:
# qcut 을 통해 3단계로 "silver", "gold", "platinum" 고객군을 나눕니다. 
rfm["RFM_class"] = pd.____(rfm["RFM_score"], q=?, labels=["silver", "gold", "platinum"])
rfm.head()

In [ ]:
# "RFM_class" 별로 그룹화 하고 "RFM_score" 의 describe 값을 구합니다.
rfm.groupby("RFM_class")["RFM_score"].describe()

In [ ]:
# barplot으로 RFM_class 별 평균 RFM_score 나타냅니다.
sns.barplot(data=rfm, x="RFM_class", y="RFM_score")

In [ ]:
# boxplot으로 RFM_class 별 평균 RFM_score 나타냅니다.
sns.boxplot(data=rfm, x="RFM_class", y="RFM_score")

In [ ]:
# pointplot으로 x=R, hue=RFM_class 별 평균 y=RFM_score 나타냅니다.
# hue 옵션을 사용하면 특정 컬럼을 지정해서 표기할 수 있습니다.
sns.pointplot(data=rfm, x="R", y="RFM_score", hue="RFM_class")

In [ ]:
# pointplot으로 x=F, hue=RFM_class 별 평균 y=RFM_score 나타냅니다.
sns.pointplot(data=rfm, x="F", y="RFM_score", hue="RFM_class")

In [ ]:
# pointplot으로 x=M, hue=RFM_class 별 평균 y=RFM_score 나타냅니다.
sns.pointplot(data=rfm, x="M", y="RFM_score", hue="RFM_class")

In [ ]:
# "RFM_class" 별로 그룹화합니다.
# "Recency", "Frequency" 의 평균을 구합니다.
# "MonetaryValue"의 "mean", "sum", "count" 값을 구합니다.
rfm_agg = rfm.groupby("RFM_class").agg({"Recency": "___", 
                              "Frequency": "___",
                             "MonetaryValue": ["___", "___", "___" ]})

In [ ]:
# 위에서 구한 값을 .style.background_gradient() 통해서 시각화합니다.
rfm_agg.style.background_gradient()

In [ ]:
#.style.format("{:,}") 로 천단위에 콤마(,)표기 하도록 문자열 포맷을 지정합니다.
rfm_agg.astype(int).style.format("{:,}")

In [ ]:
# pairplot 을 통해 "RFM_class" 별로 분포를 시각화 합니다.


In [ ]:
# .corr() 함수로 변수간 상관관계(선형적 관계)를 분석합니다.
# 1에 가까울 수록 양의 상관관계, -1에 가까울 수록 음의 상관관계를 나타냅니다.
corr = rfm.____()
corr

In [ ]:
# 삼각형태의 heatmap 형태를 만들기 위해 삼각형 마스크를 만듭니다.
mask = np.triu(np.ones_like(corr))

In [ ]:
# 상관관계를 열분포 형태의 이미지로 보여주는 heatmap 으로 데이터 시각화합니다.



In [ ]:
# RFM 분석 자료를 csv 로 저장합니다.


In [ ]:
# 저장된 csv 형태의 RFM 분석 자료를 읽어옵니다. 
